# 2a. Create basic network

In [ ]:
# Standard library and path imports
import set_path
import warnings

# Third-party library imports
import pandas as pd
import shapely.ops as so
import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import folium

# Local or project-specific imports
import plot_utils
import poly_utils
import bgt_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

## Import BGT data

In [ ]:
# Get BGT data
df_bgt_full = bgt_utils.get_bgt_data_for_bbox(st.bbox, st.bgt_layers)
df_bgt_full['naam'].value_counts()

## Select pilot areas

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)

In [ ]:
# Only keep BGT data within pilot areas
df_bgt = df_bgt_full.sjoin(df_areas, how='inner', predicate='within') # note: only sidewalk polygons fully inside area are included
df_bgt['naam_left'].value_counts()

## Visualize BGT data

In [ ]:
# set True for satellite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add sidewalks
folium.GeoJson(data=df_bgt[df_bgt['naam_left'] == 'voetpad'], style_function=lambda x: {"fillColor": "orange"}).add_to(map)
folium.GeoJson(data=df_bgt[df_bgt['naam_left'] == 'inrit'], style_function=lambda x: {"fillColor": "blue"}).add_to(map)
folium.GeoJson(data=df_bgt[df_bgt['naam_left'] == 'voetgangersgebied'], style_function=lambda x: {"fillColor": "purple"}).add_to(map)

map

## Pre-process BGT data

In [ ]:
# Remove obstacles (small interiors)
df_bgt['geometry_no_holes'] = df_bgt.progress_apply(  
   lambda row: poly_utils.remove_interiors(row.geometry, st.min_interior_size), axis=1)
df_bgt = df_bgt.set_geometry('geometry_no_holes')

# Merge sidewalk polygons
df_bgt = GeoDataFrame(geometry=gpd.GeoSeries(df_bgt['geometry_no_holes'].unary_union))
df_bgt = gpd.GeoDataFrame(df_bgt.geometry.explode()) 

# Ignore sidewalk polygons that are too small
df_bgt['area'] = df_bgt['geometry'].area
df_bgt = df_bgt[df_bgt.area > st.min_area_size]

## Create centerlines

In [ ]:
# Calculate centerlines
df_bgt['centerlines'] = df_bgt.progress_apply(
   lambda row: Centerline(row.geometry, interpolation_distance=0.5), axis=1)
df_bgt = df_bgt.set_geometry('centerlines')

In [ ]:
# Merge lines
df_bgt['centerlines'] = df_bgt['centerlines'].progress_apply(so.linemerge)

# Add sidewalk ID
df_bgt['sidewalk_id'] = range(0, len(df_bgt))

# Get each centerline in separate row
df_bgt_exp = df_bgt.explode(ignore_index=True)
df_bgt_exp.reset_index(inplace=True, names='cl_id')

## Cut lines that are too long

In [ ]:
# Remove geometry column and add length column
df_bgt_cut = df_bgt_exp.drop('geometry', axis=1)
df_bgt_cut['length'] = df_bgt_cut['centerlines'].length

# Cut linestrings longer than a max length (this takes a while)
df_bgt_cut = poly_utils.shorten_linestrings(df_bgt_cut, st.max_ls_length)

# Add shortened linestrings to previous dataframe
df_bgt_exp = pd.merge(df_bgt_exp.drop(['centerlines'], axis=1), df_bgt_cut.drop(['area', 'sidewalk_id'], axis=1), 
                                                            how = 'outer', on = 'cl_id')

## Store

In [ ]:
# Finalize dataframe
df_bgt_exp = df_bgt_exp.set_crs(st.CRS)

df_bgt_final = df_bgt_exp[['cl_id', 'centerlines', 'length', 'sidewalk_id']]  
df_bgt_final.head(3)

In [ ]:
# Write basic network to file
df_bgt_final.to_file(cf.output_basic_network, driver='GPKG')

In [ ]:
# Create dataframe related sidewalks (to be used for widths)
df_bgt_sidewalks = df_bgt_exp[['sidewalk_id', 'geometry', 'area']]
df_bgt_sidewalks = df_bgt_sidewalks.drop_duplicates().reset_index(drop=True)
df_bgt_sidewalks = df_bgt_sidewalks.set_geometry('geometry')
df_bgt_sidewalks = df_bgt_sidewalks.set_crs(st.CRS)

In [ ]:
# Write sidewalks related to basic network to file
df_bgt_sidewalks.to_file(cf.output_sidewalks_basic_network, driver='GPKG')

## Visualize BGT data and basic network

In [ ]:
# set True for satellite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add areas
folium.GeoJson(data=df_areas, style_function=lambda x: {"fillColor": "orange"}).add_to(map)    

# Add sidewalks
folium.GeoJson(data=df_bgt_sidewalks, style_function=lambda x: {"fillColor": "grey"}).add_to(map)

# Add basic network
folium.GeoJson(data=df_bgt_final, style_function=lambda x: {"color": "black"}).add_to(map)

map

In [ ]:
# Add title
loc = 'Basic network'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)

map.get_root().html.add_child(folium.Element(title_html))

In [ ]:
# Store map
map.save(cf.basic_network_map)